# Cleaning Income Tax Data

In [1]:
import pandas as pd
import urllib
import os

2004 to 2008 statistics were based on a stratified random sample of individual income tax and benefit returns. However, beginning with the 2009 tax year, Final Statistics contains data based on all of the returns filed. The most recent assessment is used in the compilation of the statistics.

In [10]:
tax_by_income_url = {2004: "http://www.cra-arc.gc.ca/gncy/stts/gb04/pst/fnl/csv/table2.csv",
                     2005: "http://www.cra-arc.gc.ca/gncy/stts/gb05/pst/fnl/csv/table2.csv",
                     2006: "http://www.cra-arc.gc.ca/gncy/stts/gb06/pst/fnl/csv/t02ca.csv",
                     2007: "http://www.cra-arc.gc.ca/gncy/stts/gb07/pst/fnl/csv/t02ca.csv",
                     2008: "http://www.cra-arc.gc.ca/gncy/stts/gb08/pst/fnl/csv/t02ca.csv",
                     2009: "http://www.cra-arc.gc.ca/gncy/stts/gb09/pst/fnl/csv/t02ca.csv",
                     2010: "http://www.cra-arc.gc.ca/gncy/stts/gb10/pst/fnl/csv/t02ca.csv",
                     2011: "http://www.cra-arc.gc.ca/gncy/stts/gb11/pst/fnl/csv/t02ca.csv",
                     2012: "http://www.cra-arc.gc.ca/gncy/stts/prlmnry/2012/tbl2-eng.csv"} # preliminary data

column_dictionary_04_08 = 'http://www.cra-arc.gc.ca/gncy/stts/gb04/pst/fnl/tb2_f1-f7-eng.html'

## Preliminary cleaning

In [11]:
def delete_extra_cols(df):
    columns2delete = [col for col in df.columns if '_r' in col or '_l' in col or 'Unnamed' in col] 
    for col in columns2delete: del df[col]
    return df

def clean_the_df(rough_df, header_lines):
    length_of_dataframe = header_lines[1] - 1
    temp_filenames = []
    for n, i in enumerate(header_lines):
        filename = '../data/test_' + str(n) + '.csv'
        if n == 0:
            rough_df[i:length_of_dataframe].to_csv(filename, index=False)
        else:
            rough_df[i-1:i+length_of_dataframe].to_csv(filename, header=False, index=False)
        temp_filenames.append(filename)

    for n, filename in enumerate(temp_filenames):
        if n == 0: clean_df = pd.DataFrame.from_csv(filename, index_col=None)
        else: clean_df = clean_df.join(pd.DataFrame.from_csv(filename, index_col=None), 
                                   lsuffix='_l' + str(n), rsuffix='_r' + str(n))
        os.remove(filename)
    
    return clean_df

In [12]:
def write_online_table_to_file(url, datafile_name):
    page = urllib.urlopen(url).read()
    f = open(datafile_name, 'w')
    f.write(page)
    f.close()
    return datafile_name

## Save the online tables to files

#### 2012 data (preliminary)

In [5]:
write_online_table_to_file(tax_by_income_url[2012], "../data/raw_tax_data_2012.csv")
df12 = pd.DataFrame.from_csv("../data/raw_tax_data_2012.csv", index_col=None)
df12.to_csv('../data/cleaned_tax_data_2012.csv')

#### 2011 data

In [6]:
write_online_table_to_file(tax_by_income_url[2011], "../data/raw_tax_data_2011.csv")
df11 = pd.DataFrame.from_csv("../data/raw_tax_data_2011.csv", index_col=None)
df11.to_csv('../data/cleaned_tax_data_2011.csv')

#### 2010 data

In [7]:
write_online_table_to_file(tax_by_income_url[2010], "../data/raw_tax_data_2010.csv")
rough_df = pd.DataFrame.from_csv("../data/raw_tax_data_2010.csv", index_col=None)

In [8]:
header_lines = rough_df.query("code2 == '1'").index
df10 = clean_the_df(rough_df, header_lines)
df10['Item'] = df10['item_e_r1']
df10 = delete_extra_cols(df10)
df10.to_csv('../data/cleaned_tax_data_2010.csv')

#### 2009 data

In [9]:
write_online_table_to_file(tax_by_income_url[2009], "../data/raw_tax_data_2009.csv")
rough_df = pd.DataFrame.from_csv("../data/raw_tax_data_2009.csv", index_col=None)

In [10]:
header_lines = rough_df.query("CODE2 == '1'").index
df09 = clean_the_df(rough_df, header_lines)
df09['Item'] = df09['iteme_r1']
df09 = delete_extra_cols(df09)
df09.to_csv('../data/cleaned_tax_data_2009.csv')

#### 2004 - 2008 data

In [16]:
def save_rough_files(year):
    filename = "../data/raw_tax_data_" + str(year) + ".csv"
    write_online_table_to_file(tax_by_income_url[year], filename)
    return filename

In [17]:
f_names = {}
for year in [2004, 2005, 2006, 2007, 2008]:
    f_names[year] = save_rough_files(year)

In [19]:
dfd = {}
for year in f_names:
    dfd[year] = pd.DataFrame.from_csv(f_names[year], index_col=None)

In [22]:
dfd[2004].columns

Index([u'F1', u'F2', u'F3', u'F4', u'F5', u'F6', u'CODE1', u'CODE2', u'prove', u'provf', u'taxyr', u'f7'], dtype='object')

#### Get a dictionary for the columns

In [146]:
import col_dict_2004_08

In [137]:
col_dict = {}
for row in col_dict_2004_08.col_dict:
    f7_val = row[0]
    for i, x in enumerate(row[1:]):
        f_col = col_dict_2004_08.list_order[i+1]
        col_dict[(f7_val, f_col)] = x

In [138]:
def flatten_df(stacked_df):
    data_cols = ['F1','F2','F3','F4','F5','F6']
    df = pd.DataFrame(index=range(len(set(stacked_df['CODE2']))))
    df['tax_year'] = pd.Series(index=df.index)
    for i in stacked_df.index:
        f7_val = stacked_df['f7'][i]
        item_num = stacked_df['CODE2'][i]
        tax_year = stacked_df['taxyr'][i]
        df.ix[item_num-1, 'tax_year'] = tax_year
        for c in data_cols:
            d = stacked_df[c][i]
            key = (str(f7_val), c.lower())
            if key == ('8', 'f5'): break
            new_col = col_dict[key]
            new_col = new_col.replace('$', '')
            if new_col not in df.columns: df[new_col] = pd.Series(index=df.index)
            df.ix[item_num-1, new_col] = d
    return df

In [139]:
flat_df = {}
for year in dfd: flat_df[year] = flatten_df(dfd[year])

In [140]:
def get_item_dict(df):
    item_dict = {}
    for i in df.index:
        item_dict[df['CODE2'][i]] = df['ITEME'][i]
    return item_dict

In [141]:
item_dict = {}
for year in [2005, 2006, 2007, 2008]:
    item_dict[year] = get_item_dict(dfd[year])

I semi-manually assembled this dictionary from an html table: http://www.cra-arc.gc.ca/gncy/stts/gb04/pst/fnl/tb2-5-eng.html

In [142]:
import item_dict_2004
item_dict[2004] = {}
for pair in item_dict_2004.item_dict:
    item_dict[2004][pair[1]] = pair[0]

In [143]:
for year in flat_df:
    flat_df[year]['item'] = pd.Series(index=flat_df[year].index)
    for i in flat_df[year].index:
        flat_df[year].ix[i, 'item'] = item_dict[year][i+1]

In [147]:
dfo = pd.concat([flat_df[year] for year in flat_df], ignore_index=True)

In [149]:
dfo.to_csv("../data/partially_clean_04_08.csv")

## Save point 1

In [3]:
dfo = pd.DataFrame.from_csv("../data/partially_clean_04_08.csv")
df09 = pd.DataFrame.from_csv('../data/cleaned_tax_data_2009.csv')
df10 = pd.DataFrame.from_csv('../data/cleaned_tax_data_2010.csv')
df11 = pd.DataFrame.from_csv('../data/cleaned_tax_data_2011.csv')
df12 = pd.DataFrame.from_csv('../data/cleaned_tax_data_2012.csv')

## Standardize the column names

In [4]:
df12.columns

Index([u'#', u'Item', u'Poste', u'Tax year/D'ann�e d'imposition', u' Grand total/Total global #', u'Grand total/Total global $ (000)', u'4999 and under/Moins de 4 999 #', u'4999 and under/Moins de 4 999 $ (000)', u'5000 - 9999 #', u'5000 - 9999 $ (000)', u'10000 - 14999 #', u'10000 - 14999 $ (000)', u'15000 - 19999 #', u'15000 - 19999 $ (000)', u'20000 - 24999 #', u'20000 - 24999 $ (000)', u'25000 - 29999 #', u'25000 - 29999 $ (000)', u'30000 - 34999 #', u'30000 - 34999 $ (000)', u'35000 - 39999 #', u'35000 - 39999 $ (000)', u'40000 - 44999 #', u'40000 - 44999 $ (000)', u'45000 - 49999 #', u'45000 - 49999 $ (000)', u'50000 - 54999 #', u'50000 - 54999 $ (000)', u'55000 - 59999 #', u'55000 - 59999 $ (000)', u'60000 - 69999 #', u'60000 - 69999 $ (000)', u'70000 - 79999 #', u'70000 - 79999 $ (000)', u'80000 - 89999 #', u'80000 - 89999 $ (000)', u'90000 - 99999 #', u'90000 - 99999 $ (000)', u'100000 - 149999 #', u'100000 - 149999 $ (000)', u'150000 - 249999 #', u'150000 - 249999 $ (000)', u

In [5]:
for c in df11.columns:
    if c not in df12.columns: print c

50000 and over/et plus #
50000 and over/et plus $ (000)


In [6]:
for c in df12.columns:
    if c not in df11.columns: print c

In [7]:
df11.columns

Index([u'#', u'Item', u'Poste', u'Tax year/D'ann�e d'imposition', u' Grand total/Total global #', u'Grand total/Total global $ (000)', u'4999 and under/Moins de 4 999 #', u'4999 and under/Moins de 4 999 $ (000)', u'5000 - 9999 #', u'5000 - 9999 $ (000)', u'10000 - 14999 #', u'10000 - 14999 $ (000)', u'15000 - 19999 #', u'15000 - 19999 $ (000)', u'20000 - 24999 #', u'20000 - 24999 $ (000)', u'25000 - 29999 #', u'25000 - 29999 $ (000)', u'30000 - 34999 #', u'30000 - 34999 $ (000)', u'35000 - 39999 #', u'35000 - 39999 $ (000)', u'40000 - 44999 #', u'40000 - 44999 $ (000)', u'45000 - 49999 #', u'45000 - 49999 $ (000)', u'50000 - 54999 #', u'50000 - 54999 $ (000)', u'55000 - 59999 #', u'55000 - 59999 $ (000)', u'60000 - 69999 #', u'60000 - 69999 $ (000)', u'70000 - 79999 #', u'70000 - 79999 $ (000)', u'80000 - 89999 #', u'80000 - 89999 $ (000)', u'90000 - 99999 #', u'90000 - 99999 $ (000)', u'100000 - 149999 #', u'100000 - 149999 $ (000)', u'150000 - 249999 #', u'150000 - 249999 $ (000)', u

In [8]:
def clean_2011_2012_dfs(df):
    df["total #"] = df[' Grand total/Total global #']
    df["total $"] = df['Grand total/Total global $ (000)']
    df["<4999 #"] = df['4999 and under/Moins de 4 999 #']
    df["<4999 $"] = df['4999 and under/Moins de 4 999 $ (000)']
    df['>250000 #'] = df['250000 and over/et plus #']
    df['>250000 $'] = df['250000 and over/et plus $ (000)']
    return df

In [9]:
df11 = clean_2011_2012_dfs(df11)
df12 = clean_2011_2012_dfs(df12)

In [10]:
df10.columns

Index([u'Grand total/Total global #', u'Grand total/Total global $', u'Loss and nil/Perte et n�ant #', u'Loss and nil/Perte et n�ant $', u'1 - 4999 #', u'1 - 4999 $', u'5000 - 9999 #', u'5000 - 9999 $', u'10000 - 14999 #', u'10000 - 14999 $', u'15000 - 19999 #', u'15000 - 19999 $', u'20000 - 24999 #', u'20000 - 24999 $', u'25000 - 29999 #', u'25000 - 29999 $', u'30000 - 34999 #', u'30000 - 34999 $', u'35000 - 39999 #', u'35000 - 39999 $', u'40000 - 44999 #', u'40000 - 44999 $', u'45000 - 49999 #', u'45000 - 49999 $', u'50000 - 54999 #', u'50000 - 54999 $', u'55000 - 59999 #', u'55000 - 59999 $', u'60000 - 69999 #', u'60000 - 69999 $', u'70000 - 79999 #', u'70000 - 79999 $', u'80000 - 89999 #', u'80000 - 89999 $', u'90000 - 99999 #', u'90000 - 99999 $', u'100000 - 149999 #', u'100000 - 149999 $', u'150000 - 249999 #', u'150000 - 249999 $', u'250000 and over/et plus #', u'250000 and over/et plus $', u'50000 and over/et plus #', u'50000 and over/et plus $', u'Item'], dtype='object')

In [11]:
df10['total #'] = df10['Grand total/Total global #']
df10['total $'] = df10['Grand total/Total global $']
df10['<4999 #'] = df10[df10.columns[2]] + df10['1 - 4999 #']
df10['<4999 $'] = df10[df10.columns[3]] + df10['1 - 4999 $']
df10['>250000 #'] = df10['250000 and over/et plus #']
df10['>250000 $'] = df10['250000 and over/et plus $']

In [12]:
df09.columns

Index([u'Grand total #/Total global #', u'Grand total $/Total global $', u'Loss and nil #/Perte et n�ant # ', u'Loss and nil $/Perte et n�ant $', u'1 - 4999 #', u'1 - 4999 $', u'5000 - 9999 #', u'5000 - 9999 $', u'10000 - 14999 # ', u'10000 - 14999 $', u'15000 - 19999 #', u'15000 - 19999 $', u'20000 - 24999 #', u'20000 - 24999 $', u'25000 - 29999 #', u'25000 - 29999 $', u'30000 - 34999 #', u'30000 - 34999 $', u'35000 - 39999 #', u'35000 - 39999 $40000 - 44999 #', u'40000 - 44999 #', u'40000 - 44999 $', u'45000 - 49999 #', u'45000 - 49999 $', u'50000 - 54999 #', u'50000 - 54999 $', u'55000 - 59999 #', u'55000 - 59999 $', u'60000 - 69999 #', u'60000 - 69999 $', u'70000 - 79999 #', u'70000 - 79999 $', u'80000 - 89999 #', u'80000 - 89999 $', u'90000 - 99999 #', u'90000 - 99999 $', u'100000 - 149999 #', u'100000 - 149999 $', u'150000 - 249999 #', u'150000 - 249999 $', u'250000 and over #/250 000 et plus #', u'250000 and over $/250 000 et plus $', u'CODE1', u'iteme', u'itemf', u'CODE2', u'It

In [13]:
df09['35000 - 39999 $'] = df09['35000 - 39999 $40000 - 44999 #']
df09['10000 - 14999 #'] = df09['10000 - 14999 # ']
df09['total #'] = df09['Grand total #/Total global #']
df09['total $'] = df09['Grand total $/Total global $']
df09['<4999 #'] = df09[df09.columns[2]] + df09['1 - 4999 #']
df09['<4999 $'] = df09[df09.columns[3]] + df09['1 - 4999 $']
df09['>250000 #'] = df09['250000 and over #/250 000 et plus #']
df09['>250000 $'] = df09['250000 and over $/250 000 et plus $']

In [14]:
dfo.columns

Index([u'tax_year', u'Loss and nil (number)', u'Loss and nil(amount)', u'1 - 10,000 (number)', u'1 - 10,000 (amount)', u'10,000 - 15,000 (number)', u'10,000 - 15,000 (amount)', u'15,000 - 20,000 (number)', u'15,000 - 20,000 (amount)', u'20,000 - 25,000 (number)', u'20,000 - 25,000 (amount)', u'25,000 - 30,000 (number)', u'25,000 - 30,000 (amount)', u'30,000 - 35,000 (number)', u'30,000 - 35,000 (amount)', u'35,000 - 40,000 (number)', u'35,000 - 40,000 (amount)', u'30,000 - 40,000 (number)', u'30,000 - 40,000 (amount)', u'40,000 - 45,000 (number)', u'40,000 - 45,000 (amount)', u'45,000 - 50,000 (number)', u'45,000 - 50,000 (amount)', u'40,000 - 50,000 (number)', u'40,000 - 50,000 (amount)', u'50,000 - 60,000 (number)', u'50,000 - 60,000 (amount)', u'60,000 - 70,000 (number)', u'60,000 - 70,000 (amount)', u'70,000 - 80,000 (number)', u'70,000 - 80,000 (amount)', u'80,000 - 90,000 (number)', u'80,000 - 90,000 (amount)', u'90,000 - 100,000 (number)', u'90,000 - 100,000 (amount)', u'50,000 

In [15]:
del dfo['50,000 and over (number)']
del dfo['50,000 and over(amount)']
del dfo['50,000 - 100,000 (number)']
del dfo['50,000 - 100,000 (amount)']
del dfo['Grand Total (number)']
del dfo['Grand total (amount)']
del dfo['30,000 - 40,000 (number)']
del dfo['30,000 - 40,000 (amount)']
del dfo['40,000 - 50,000 (number)']
del dfo['40,000 - 50,000 (amount)']

In [16]:
for c in dfo.columns:
    c2 = c.replace('Loss and nil(amount)', 'Loss and nil (amount)')
    c2 = c2.replace('250,000 and over', '>250,000')
    c2 = c2.replace('(amount)', '$')
    c2 = c2.replace('(number)', '#')
    c2 = c2.replace(',', '')
    c2 = c2.replace('Loss and nil', '<0')
    dfo[c2] = dfo[c]
    if c != c2: del dfo[c]

Create a tax year column so we can combine into a single table

In [17]:
df12["tax_year"] = pd.Series([2012]*len(df12), index=df12.index)
df11["tax_year"] = pd.Series([2011]*len(df11), index=df11.index)
df10["tax_year"] = pd.Series([2010]*len(df10), index=df10.index)
df09["tax_year"] = pd.Series([2009]*len(df09), index=df09.index)

After examining the datafiles, it appears that the dollar columns in all files are in thousands, even though the 2011 file is the only one that says this explicitly. I will remove the ' (000)' from the column titles for the 2011 file, and modify the dollar amounts later.

In [18]:
def remove_thousands(df):
    for c in df.columns:
        if c.find('$') != -1:
            df[c[:-6]] = df[c]
    return df

In [19]:
df11 = remove_thousands(df11)
df12 = remove_thousands(df12)

In [20]:
common_cols = set(df12.columns).intersection(set(df11.columns)).intersection(set(df10.columns)).intersection(set(df09.columns))

In [21]:
clean_cols = []
clean_cols.append('item')
clean_cols.append('type')
clean_cols.append('tax_year')
for c in common_cols:
    if '#' in c: clean_cols.append(c[:-2])

In [22]:
half_n_rows = len(df12) + len(df11) + len(df10) + len(df09)

In [23]:
df_master = pd.DataFrame(columns=clean_cols, index=range(2*half_n_rows))

In [24]:
for c in df_master.columns:
    if c == 'item':
        df_master['item'] = (list(df12['Item']) + list(df11['Item']) + list(df10['Item']) + list(df09['Item']))*2
    elif c == 'type':
        df_master['type'] = ['#']*half_n_rows + ['$']*half_n_rows
    elif c == 'tax_year':
        df_master['tax_year'] = (list(df12['tax_year']) + list(df11['tax_year']) + list(df10['tax_year']) + list(df09['tax_year']))*2
    else:
        df_master[c] = list(df12[c + ' #']) + list(df11[c + ' #']) + list(df10[c + ' #']) + list(df09[c + ' #']) \
        + list(df12[c + ' $']) + list(df11[c + ' $']) + list(df10[c + ' $']) + list(df09[c + ' $'])

Now for 2004 - 2008

In [25]:
new_cols = ['item', 'type', 'tax_year']
for c in dfo.columns:
    if '#' in c: new_cols.append(c[:-2])

In [26]:
dfo2 = pd.DataFrame(columns=new_cols, index=range(2*len(dfo)))

In [27]:
for c in dfo2.columns:
    if c == 'item':
        dfo2[c] = list(dfo[c]) * 2
    elif c == 'tax_year':
        dfo2[c] = [int(x) for x in list(dfo[c])] * 2
    elif c == 'type':
        dfo2[c] = ['#']*len(dfo) + ['$']*len(dfo)
    else:
        dfo2[c] = list(dfo[c + ' #']) + list(dfo[c + ' $'])

## Making the column headings more readable

In [37]:
col_labels = {
"<4999": "< $5k",
"5000 - 9999": "$5k - 10k",
"10000 - 14999": "$10k - 15k",
"15000 - 19999": "$15k - 20k",
"20000 - 24999": "$20k - 25k",
"25000 - 29999": "$25k - 30k",
"30000 - 34999": "$30k - 35k",
"35000 - 39999": "$35k - 40k",
"40000 - 44999": "$40k - 45k",
"45000 - 49999": "$45k - 50k",
"50000 - 54999": "$50k - 55k",
"55000 - 59999": "$55k - 60k",
"60000 - 69999": "$60k - 70k",
"70000 - 79999": "$70k - 80k",
"80000 - 89999": "$80k - 90k",
"90000 - 99999": "$90k - 100k",
"100000 - 149999": "$100k - 150k",
"150000 - 249999": "$150k - 250k",
">250000": "> $250k"}

col_labels_04_08 = {
'<0': '< $0',
"1 - 10000": "$1 - 10k",
"10000 - 15000": "$10k - 15k",
"15000 - 20000": "$15k - 20k",
"20000 - 25000": "$20k - 25k",
"25000 - 30000": "$25k - 30k",
"30000 - 35000": "$30k - 35k",
"35000 - 40000": "$35k - 40k",
"40000 - 45000": "$40k - 45k",
"45000 - 50000": "$45k - 50k",
"50000 - 60000": "$50k - 60k",
"60000 - 70000": "$60k - 70k",
"70000 - 80000": "$70k - 80k",
"80000 - 90000": "$80k - 90k",
"90000 - 100000": "$90k - 100k",
"100000 - 150000": "$100k - 150k",
"150000 - 250000": "$150k - 250k",
">250000": "> $250k"}

In [29]:
def rename_cols(df, col_label_dict):
    for col in df.columns:
        if col in col_label_dict:
            df[col_label_dict[col]] = df[col]
            del df[col]
    return df

In [30]:
df_master = rename_cols(df_master, col_labels)
dfo2 = rename_cols(dfo2, col_labels_04_08)

In [31]:
ordered_cols = ['item', 'type', 'tax_year', 'total', 
                '< $5k', '$5k - 10k', '$10k - 15k', 
                '$15k - 20k', '$20k - 25k', '$25k - 30k',
                '$30k - 35k', '$35k - 40k', '$40k - 45k',
                '$45k - 50k', '$50k - 55k', '$55k - 60k',
                '$60k - 70k', '$70k - 80k', '$80k - 90k',
                '$90k - 100k', '$100k - 150k', '$150k - 250k', 
                '> $250k']

In [32]:
df_master = df_master[ordered_cols]

## Adjust to 2015 dollars and multiply the dollar amounts by 1000

Using the december to december inflation rates from:  http://www.inflation.eu/inflation-rates/canada/historic-inflation/cpi-inflation-canada.aspx

In [33]:
inflation_rate = {2014: 1.47, 2013: 1.24, 2012: 0.83,
    2011: 2.30, 2010: 2.35, 2009: 1.32,
    2008: 1.16, 2007: 2.38, 2006: 1.67,
    2005: 2.09, 2004: 2.13}

inflation_multipliers = {}
for yr_rate in inflation_rate:
    for yr_mult in range(2004, yr_rate+1):
        if yr_mult in inflation_multipliers:
            inflation_multipliers[yr_mult] = (1. + 0.01*inflation_rate[yr_rate]) * inflation_multipliers[yr_mult]
        else:
            inflation_multipliers[yr_mult] = 1.

In [34]:
def adjust_dollar_amounts(df):
    for i in df.index:
        tax_year = df.tax_year[i]
        print i,
        if df.type[i] == '$':
            for col in set(df.columns).difference(set(['item', 'type', 'tax_year'])):
                df.ix[i, col] = 1000. * inflation_multipliers[tax_year] * df[col][i]
    return df

In [35]:
df_master = adjust_dollar_amounts(df_master)
dfo2 = adjust_dollar_amounts(dfo2)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [36]:
df_master.to_csv('../data/tax_data_unclean_items.csv')
dfo2.to_csv('../data/tax_data_unclean_items_04_08.csv')

## Save point 2

In [1]:
import pandas as pd
df_master = pd.DataFrame.from_csv('../data/tax_data_unclean_items.csv')
dfo2 = pd.DataFrame.from_csv('../data/tax_data_unclean_items_04_08.csv')

## Standardize the item titles

In [2]:
print len(set(df_master['item']))
print len(set(dfo2['item']))

149
224


In [3]:
import item_synonyms
reload(item_synonyms)

<module 'item_synonyms' from 'item_synonyms.pyc'>

In [4]:
synonym_dict = {}
for li in item_synonyms.synonyms:
    if len(li) > 1:
        for item in li:
            synonym_dict[item] = li[0]

In [5]:
def clean_item_names(df, synonym_dict):
    for i in df.index:
        if df.item[i] in synonym_dict.keys():
            df.ix[i, 'item'] = synonym_dict[df.item[i]]
    return df

In [6]:
df_master = clean_item_names(df_master, synonym_dict)
dfo2 = clean_item_names(dfo2, synonym_dict)

In [7]:
dfo2.to_csv('../data/all_clean_tax_data_04_08.csv')
df_master.to_csv('../data/all_clean_tax_data.csv')

### Save point 3 

In [8]:
import pandas as pd
df_master = pd.DataFrame.from_csv('../data/all_clean_tax_data.csv')
dfo2 = pd.DataFrame.from_csv('../data/all_clean_tax_data_04_08.csv')

### Let's try combining the 50-55k and 55-60k columns for the 2009-12 data to match up better with 2004-08

In [9]:
df_master.columns

Index([u'item', u'type', u'tax_year', u'total', u'< $5k', u'$5k - 10k', u'$10k - 15k', u'$15k - 20k', u'$20k - 25k', u'$25k - 30k', u'$30k - 35k', u'$35k - 40k', u'$40k - 45k', u'$45k - 50k', u'$50k - 55k', u'$55k - 60k', u'$60k - 70k', u'$70k - 80k', u'$80k - 90k', u'$90k - 100k', u'$100k - 150k', u'$150k - 250k', u'> $250k'], dtype='object')

In [10]:
df_master['$50k - 60k'] = (df_master['$50k - 55k'] + df_master['$55k - 60k'])

In [11]:
del df_master['$50k - 55k']
del df_master['$55k - 60k']

In [12]:
new_ordered_cols = ['item', 'type', 'tax_year', 'total', 
                '< $5k', '$5k - 10k', '$10k - 15k', 
                '$15k - 20k', '$20k - 25k', '$25k - 30k',
                '$30k - 35k', '$35k - 40k', '$40k - 45k',
                '$45k - 50k', '$50k - 60k',
                '$60k - 70k', '$70k - 80k', '$80k - 90k',
                '$90k - 100k', '$100k - 150k', '$150k - 250k', 
                '> $250k']

In [13]:
df_master = df_master[new_ordered_cols]

In [14]:
df_master.to_csv('../data/all_clean_tax_data_fewer_cols.csv')

### Try to fix some of the item naming problems

In [15]:
old_tax_yrs = dfo2.query("type == '$' and item == 'CPP or QPP benefits'").tax_year

In [16]:
its = list(set(df_master.item).union(set(dfo2.item)))
its.sort()
for item in its[:10]:
    new_tax_yrs = list(df_master.query("type == '$' and item == @item").tax_year)
    old_tax_yrs = list(dfo2.query("type == '$' and item == @item").tax_year)
    new_tax_yrs.sort()
    old_tax_yrs.sort()
    print item, old_tax_yrs + new_tax_yrs

CPP contributions on self-employment [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
CPP or QPP benefits [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
CPP or QPP contributions [2004, 2005, 2006, 2007, 2008]
CPP or QPP contributions on employment [2009, 2010, 2011, 2012]
CPP or QPP contributions on self-employment [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
Canada employment amount [2006, 2007, 2008, 2009, 2010, 2011, 2012]
Canadian Forces personnel and police deduction [2009, 2010, 2011, 2012]
EI benefits [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
EI premiums [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012]
EI premiums on self-employment and other eligible earnings [2010, 2011, 2012]


### Deleteing problem rows

In [17]:
# 2004, social benefits repayment
def drop_row(item, year, df):
    indexes_2_drop = df.query("item == @item and tax_year == @year").index
    df = df.drop(indexes_2_drop)
    return df

In [18]:
dfo2 = drop_row('social benefits repayment', 2004, dfo2)
dfo2 = drop_row('Federal Tax', 2004, dfo2)
dfo2.index = range(len(dfo2))

In [27]:
import numpy as np
x = np.nan

In [42]:
val_cols = set(df_master.columns).difference(set(['item', 'type', 'tax_year', 'total']))
nan_rows = set()
for c in val_cols:
    nan_rows = nan_rows.union(set(list(df_master[c].index[df_master[c].apply(np.isnan)])))

In [43]:
nan_rows

{15,
 48,
 51,
 54,
 72,
 98,
 99,
 102,
 103,
 108,
 109,
 438,
 439,
 440,
 453,
 486,
 489,
 492,
 510,
 536,
 537,
 540,
 541,
 546,
 547}

After checking these out, it appears all the nans should be zeros. But this is how nans are represented in the web page anyway, so no changes are necessary.

### Converting to JSON

In [50]:
def write_df_to_json(df, open_file, last_write=True):
    for i in df.index:
        row_str = '{'
        for j, c in enumerate(df.columns):
            row_str += '"' + c + '"' + ': ' + '"' + str(df[c][i]) 
            if j+1 == len(df.columns): row_str += '"'
            else: row_str += '", '
        if i+1 == len(df) and last_write: row_str += '}\n'
        else: row_str += '},\n'
        open_file.write(row_str)

In [51]:
f = open('../data/data.json', 'w')
f.write('[')
write_df_to_json(df_master, f, last_write=False)
write_df_to_json(dfo2, f, last_write=True)
f.write(']')
f.close()